In [1]:
!python --version

Python 3.12.2


In [2]:
#importing necessary modules 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import seaborn as sns
import copairs
import pycytominer
from copairs.map import average_precision
from copairs.map import mean_average_precision
from utils import * 

%load_ext autoreload
%autoreload 2

c:\Users\ssivagur\Anaconda3\envs\copairs\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
pip show copairs

Name: copairs
Version: 0.4.1
Summary: Find pairs and compute metrics between them
Home-page: 
Author: 
Author-email: John Arevalo <johnarevalo@gmail.com>
License: 
Location: c:\Users\ssivagur\Anaconda3\envs\copairs\Lib\site-packages
Requires: pandas, statsmodels, tqdm
Required-by: 
Note: you may need to restart the kernel to use updated packages.


#### Reading the dataframes 

In [4]:
#Reading the normalized and feature selected files for copairs - fixed cells and 48h time point
dict_to_load_feature_selected = {'c:\\Users\\ssivagur\\Documents\\Projects\\Cell_painting_projects\\CDoT\\Batch3_analysis_redo\\UpdatedCopairsVersion\\BR00122250_normalized_feature_select_negcon_batch.csv':'Standard CP',
                                 'c:\\Users\\ssivagur\\Documents\\Projects\\Cell_painting_projects\\CDoT\\Batch3_analysis_redo\\UpdatedCopairsVersion\\BR00122246_normalized_feature_select_negcon_batch.csv':'CP + MitoBrilliant',
                                 'c:\\Users\\ssivagur\\Documents\\Projects\\Cell_painting_projects\\CDoT\\Batch3_analysis_redo\\UpdatedCopairsVersion\\BR00122247_normalized_feature_select_negcon_batch.csv':'CP + Phenovue phalloidin 400LS',
                                 'c:\\Users\\ssivagur\\Documents\\Projects\\Cell_painting_projects\\CDoT\\Batch3_analysis_redo\\UpdatedCopairsVersion\\BR00122248_normalized_feature_select_negcon_batch.csv':'Standard CP (exposed to ChromaLive)',
                                 'c:\\Users\\ssivagur\\Documents\\Projects\\Cell_painting_projects\\CDoT\\Batch3_analysis_redo\\UpdatedCopairsVersion\\BR00122249_normalized_feature_select_negcon_batch_wo_phasefeatures.csv':'ChromaLive + Hoechst'

}


#### Overall mAP

### Sister compounds eliminated when calculating mAP

In [5]:
mAP_treatment_plot, mAP_treatment_dict = copairs_batches_newerversion(dict_to_load_feature_selected, negdiffby_parameters= ['Metadata_broad_sample', 'Metadata_MoA'], filename='Distinctiveness')

c:\Users\ssivagur\Documents\GitHub\2022_09_07_New_phenotypic_dye_testing_CDoT_Broad_Analysis\notebooks\utils.py:53: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Metadata_control_type'].fillna('trt', inplace=True)
c:\Users\ssivagur\Documents\GitHub\2022_09_07_New_phenotypic_dye_testing_CDoT_Broad_Analysis\notebooks\utils.py:54: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will nev

### Sister compounds not eliminated when calculating mAP

In [6]:
mAP_treatment_plot_in_sis, mAP_treatment_dict_in_sis = copairs_batches_newerversion(dict_to_load_feature_selected, negdiffby_parameters= ['Metadata_broad_sample'], filename='Distinctiveness')

c:\Users\ssivagur\Documents\GitHub\2022_09_07_New_phenotypic_dye_testing_CDoT_Broad_Analysis\notebooks\utils.py:53: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



c:\Users\ssivagur\Documents\GitHub\2022_09_07_New_phenotypic_dye_testing_CDoT_Broad_Analysis\notebooks\utils.py:54: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are

### Comparing the mAP with and without sister compounds

In [23]:
combined_df = pd.DataFrame()
colors = ['#0173b2','#de8f05','#029e73','#d55e00','#cc78bc']  
for i in mAP_treatment_dict.keys():
    df = mAP_treatment_dict.get(i)
    combined_df = pd.concat([combined_df, df.assign(dataset = i)])

    df = combined_df

combined_df_sis = pd.DataFrame()
for j in mAP_treatment_dict_in_sis.keys():
    df_sis = mAP_treatment_dict_in_sis.get(j)
    combined_df_sis = pd.concat([combined_df_sis, df_sis.assign(dataset = j)])

    df_sis = combined_df_sis


In [37]:

fig = go.Figure() 
custom_order = [ 'ChromaLive + Hoechst','Standard CP (exposed to ChromaLive)','CP + Phenovue phalloidin 400LS','CP + MitoBrilliant','Standard CP']

for (dataset, dataset2, color) in zip(combined_df['dataset'].unique(), combined_df_sis['dataset'].unique(), colors):
    subset_df = combined_df[combined_df['dataset'] == dataset]
    subset_df_sis = combined_df_sis[combined_df_sis['dataset'] == dataset]

      
    fig.add_trace(go.Violin(x=subset_df['mean_average_precision'], y=subset_df['dataset'],marker=dict(color=color),line=dict(color=color), box={"visible": True}, orientation='h', side='negative'))
    fig.add_trace(go.Violin(x=subset_df_sis['mean_average_precision'], y=subset_df_sis['dataset'],marker=dict(color=color),line=dict(color=color), box={"visible": True}, orientation='h', side='positive', opacity=0.5))
    fig.update_traces(points='all', width=0.5)
    fig.update_layout(height=2500, width=2750, font_family='sans serif', font=dict(size=95, color='black'), xaxis_title='Mean Average Precision', yaxis_title='', showlegend=False)
    fig.update_layout({'plot_bgcolor':'rgba(0,0,0,0)'})
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', categoryorder='array', categoryarray=custom_order)
    fig.update_xaxes(dict(range=[0, 1.02]), showline=True, linewidth=1, linecolor='black', tickangle=90)
    fig.data = fig.data[::-1] 

fig.show()